In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import requests
import json
from tqdm import tqdm

c:\Users\doha.ramadan\AppData\Local\miniconda3\envs\coding\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [69]:
ahj = pd.read_excel("D:\\CodingSystem\\assets\\AHJ_PriceList.xlsx")
ahj['SRVICE_DESCRIPTION'] = ahj['SERVICE_DESCRIPTION'].str.strip().str.upper()

print(ahj.shape)
ahj.head()

(111792, 8)


,INSURANCE_COMPANY,SERVICE_CODE,SERVICE_DESCRIPTION,PRICE,SERVICE_KEY,SERVICE_CLASSIFICATION,SERVICE_CATEGORY,SRVICE_DESCRIPTION
0,Cash,LA0013674,AFP**,300.000,75905,LAB Services,LAB-Hormones,AFP**
1,Islamic Bank,LA0013674,AFP**,300.000,75905,LAB Services,LAB-Hormones,AFP**
2,Gulf Union,LA0013674,AFP**,315.000,75905,LAB Services,LAB-Hormones,AFP**
3,MOH,LA0013674,AFP**,393.288,75905,LAB Services,LAB-Hormones,AFP**
4,AXA,LA0013674,AFP**,415.000,75905,LAB Services,LAB-Hormones,AFP**


In [70]:
ahj['SERVICE_CODE'].nunique(), ahj['SERVICE_DESCRIPTION'].nunique()

(16302, 16298)

In [71]:
sbs = pd.read_excel("D:\\CodingSystem\\assets\\SBS_Services.xlsx")
sbs['Short Description'] = sbs['Short Description'].str.strip().str.upper()
sbs['Long Description'] = sbs['Long Description'].str.strip().str.upper()
sbs['Definition'] = sbs['Definition'].fillna("").str.strip().str.upper()
sbs['Detailed Description'] = sbs['Long Description'] + "||" + sbs['Definition']

print(sbs.shape)
sbs.head()

(10081, 7)


,SBS Code,SBS Code (Hyphenated),Short Description,Long Description,Definition,Block Name,Detailed Description
0,408030000,40803-00-00,INTRACRANIAL STEREOTACTIC LOCALISATION,INTRACRANIAL STEREOTACTIC LOCALIZATION,,"Examination of skull, meninges or brain",INTRACRANIAL STEREOTACTIC LOCALIZATION||
1,409030000,40903-00-00,NEUROENDOSCOPY,NEUROENDOSCOPY,,"Examination of skull, meninges or brain",NEUROENDOSCOPY||
2,390030000,39003-00-00,CISTERNAL PUNCTURE,CISTERNAL PUNCTURE,A NEEDLE PLACED BELOW THE OCCIPITAL BONE (BACK...,Cranial tap or puncture,CISTERNAL PUNCTURE||A NEEDLE PLACED BELOW THE ...
3,390060000,39006-00-00,VENTRICULAR PUNCTURE,VENTRICULAR PUNCTURE,,Cranial tap or puncture,VENTRICULAR PUNCTURE||
4,390090000,39009-00-00,TAP FOR SUBDURAL HAEMORRHAGE,TAP FOR SUBDURAL HAEMORRHAGE,,Cranial tap or puncture,TAP FOR SUBDURAL HAEMORRHAGE||


In [72]:
sbs['SBS Code (Hyphenated)'].nunique(), sbs['Short Description'].nunique(), sbs['Long Description'].nunique(), sbs['Detailed Description'].nunique()

(10081, 10079, 10076, 10081)

In [80]:
ahj_bupa = ahj[ahj['INSURANCE_COMPANY'] == 'Bupa']
# Add new column with 'PK-' prefix removed if present:
ahj_bupa['NEW_SERVICE_DESCRIPTION'] = ahj_bupa['SERVICE_DESCRIPTION'].str.replace(r'^(PK-)+', '', regex=True)

ahj_bupa.shape

C:\Users\doha.ramadan\AppData\Local\Temp\ipykernel_16396\3550318295.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ahj_bupa['NEW_SERVICE_DESCRIPTION'] = ahj_bupa['SERVICE_DESCRIPTION'].str.replace(r'^(PK-)+', '', regex=True)


(3225, 9)

In [87]:
# Merge on NEW_SERVICE_DESCRIPTION == Long Description
merge_long = ahj_bupa.merge(
    sbs,
    how='left',
    left_on='NEW_SERVICE_DESCRIPTION',
    right_on='Long Description'
)

# Merge on NEW_SERVICE_DESCRIPTION == Short Description
merge_short = ahj_bupa.merge(
    sbs,
    how='left',
    left_on='NEW_SERVICE_DESCRIPTION',
    right_on='Short Description'
)

# Combine the two
exact_services = pd.concat([merge_long, merge_short])

# Keep only rows with a match
exact_services = exact_services[
    exact_services['Long Description'].notnull() | exact_services['Short Description'].notnull()
]

# Remove duplicates if any
exact_services = exact_services.drop_duplicates()

print(exact_services.shape)
exact_services.head()

(725, 16)


,INSURANCE_COMPANY,SERVICE_CODE,SERVICE_DESCRIPTION,PRICE,SERVICE_KEY,SERVICE_CLASSIFICATION,SERVICE_CATEGORY,SRVICE_DESCRIPTION,NEW_SERVICE_DESCRIPTION,SBS Code,SBS Code (Hyphenated),Short Description,Long Description,Definition,Block Name,Detailed Description
87,Bupa,LA0009004,ORAL GLUCOSE TOLERANCE TEST,444.11,76961,LAB Services,LAB-Biochemistry,ORAL GLUCOSE TOLERANCE TEST,ORAL GLUCOSE TOLERANCE TEST,665420020,66542-00-20,ORAL GLUCOSE TOLERANCE TEST,ORAL GLUCOSE TOLERANCE TEST,A TEST THAT MEASURES THE BODY'S RESPONSE TO SU...,Physiological assessment,ORAL GLUCOSE TOLERANCE TEST||A TEST THAT MEASU...
742,Bupa,XY0077664,RETINAL PHOTOGRAPHY OF BOTH EYES,332.74,81211,RAD Services,imaging,RETINAL PHOTOGRAPHY OF BOTH EYES,RETINAL PHOTOGRAPHY OF BOTH EYES,112180000,11218-00-00,RETINAL PHOTOGRAPHY OF BOTH EYES,RETINAL PHOTOGRAPHY OF BOTH EYES,,Other angiography,RETINAL PHOTOGRAPHY OF BOTH EYES||
906,Bupa,XY0077753,"RADIOGRAPHY OF ANKLE AND FOOT, UNILATERAL",413.98,81544,RAD Services,X-Ray,"RADIOGRAPHY OF ANKLE AND FOOT, UNILATERAL","RADIOGRAPHY OF ANKLE AND FOOT, UNILATERAL",575240401,57524-04-01,RADIOGRAPHY ANKLE & FOOT UNI,"RADIOGRAPHY OF ANKLE AND FOOT, UNILATERAL",RADIOGRAPHY OF ANKLE AND FOOT (ONE SIDE).,Radiography of lower limb,"RADIOGRAPHY OF ANKLE AND FOOT, UNILATERAL||RAD..."
910,Bupa,XY0077795,"ULTRASOUND OF FOREARM OR ELBOW, UNILATERAL",2018.77,81568,RAD Services,Ultrasound,"ULTRASOUND OF FOREARM OR ELBOW, UNILATERAL","ULTRASOUND OF FOREARM OR ELBOW, UNILATERAL",558040001,55804-00-01,U/S FOREARM/ELBOW UNI,"ULTRASOUND OF FOREARM OR ELBOW, UNILATERAL","ULTRASOUND OF FOREARM OR ELBOW (ONE SIDE), EXC...",Ultrasound of other sites,"ULTRASOUND OF FOREARM OR ELBOW, UNILATERAL||UL..."
920,Bupa,XY0077791,"RADIOGRAPHY OF HAND AND WRIST, BILATERAL",235.80,81583,RAD Services,X-Ray,"RADIOGRAPHY OF HAND AND WRIST, BILATERAL","RADIOGRAPHY OF HAND AND WRIST, BILATERAL",575120302,57512-03-02,RADIOGRAPHY HAND & WRIST BIL,"RADIOGRAPHY OF HAND AND WRIST, BILATERAL",RADIOGRAPHY OF HAND AND WRIST (BOTH SIDES) WIT...,Radiography of upper limb,"RADIOGRAPHY OF HAND AND WRIST, BILATERAL||RADI..."


In [96]:
exact_services['SERVICE_DESCRIPTION'].nunique()

725

In [92]:
different_services_df = ahj_bupa[~ahj_bupa['NEW_SERVICE_DESCRIPTION'].isin(list(exact_services['NEW_SERVICE_DESCRIPTION'].unique()))]
different_services_df = different_services_df[different_services_df['INSURANCE_COMPANY'] == 'Bupa']
different_services_df.shape

(2500, 9)

In [97]:
different_services_df['SERVICE_DESCRIPTION'].nunique()

2493

In [111]:
ahj_services = list(different_services_df['NEW_SERVICE_DESCRIPTION'].unique())
sbs_services = list(sbs['Long Description'].unique())

len(ahj_services), len(sbs_services)

(2484, 10076)

In [101]:
from transformers import AutoTokenizer
import pandas as pd

# Load your tokenizer
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")  # or any other model

# Function to count tokens
def count_tokens(text):
    if pd.isna(text):
        return 0
    return len(tokenizer.tokenize(text))

# Apply
sbs['SHORT_TOKEN_COUNT'] = sbs['Short Description'].apply(count_tokens)
sbs['LONG_TOKEN_COUNT'] = sbs['Long Description'].apply(count_tokens)

In [110]:
sbs[sbs['LONG_TOKEN_COUNT'] >= 32].shape

(332, 10)

In [112]:
# 1. Load Clinical BioBERT
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

# 2. Mask-aware mean pooling
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

# 3. Batched encoding
def encode(texts, batch_size=16):
    all_embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Encoding"):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, padding=True, truncation=True, max_length= 64, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = mean_pooling(outputs, inputs['attention_mask'])
        all_embeddings.append(embeddings)
    return torch.cat(all_embeddings, dim=0)

# 4. Encode
local_emb = encode(ahj_services)
standard_emb = encode(sbs_services)

# 5. Similarity
similarity = cosine_similarity(local_emb.cpu().numpy(), standard_emb.cpu().numpy())

# 6. Match
matches = []
for idx, sims in tqdm(enumerate(similarity), total=len(similarity), desc="Matching"):
    best_idx = sims.argmax()
    best_score = sims[best_idx]
    matches.append({
        "ahj_description": ahj_services[idx],
        "sbs_description": sbs_services[best_idx],
        "similarity_score": float(best_score)
    })

Matching: 100%|██████████| 2484/2484 [00:00<00:00, 99625.65it/s]


In [118]:
# similar_services = pd.DataFrame(matches)
# similar_services.sort_values("ahj_description").to_excel('bupa_mapped_long_services_bert.xlsx')

In [119]:
# Short description:
short_similarity = pd.read_excel("D:\\CodingSystem\\notebooks\\bupa_mapped_short_services_bert.xlsx")

# Long Description:
long_similarity = pd.read_excel("D:\\CodingSystem\\notebooks\\bupa_mapped_long_services_bert.xlsx")

short_similarity.shape, long_similarity.shape

((2484, 3), (2484, 3))

In [120]:
common_similarity = short_similarity.merge(long_similarity, how = 'inner', on = 'AHJ_DESCRIPTION')
common_similarity.shape

(2484, 5)

In [124]:
common_similarity.head()

,AHJ_DESCRIPTION,SBS_SHORT_DESCRIPTION,SHORT_SIMILARITY_SCORE,SBS_LONG_DESCRIPTION,LONG_SIMILARITY_SCORE
0,Duplex u/s extracranial/carotid & vert,DUPLEX U/S EXTRACRANIAL/CAROTID & VERT,1.000000,"DUPLEX ULTRASOUND OF EXTRACRANIAL, CAROTID AND...",0.954937
1,total arthroplasty of knee unilateral,TOTAL ARTHROPLASTY OF KNEE UNILATERAL,1.000000,"TOTAL ARTHROPLASTY OF KNEE, UNILATERAL",0.991215
2,ENDOSCOPIC BANDING OF OESOPHAGEAL VARICE,ENDOSCOPIC BANDING OF OESOPHAGEAL VARICES,0.997117,ENDOSCOPIC BANDING OF OESOPHAGEAL VARICES,0.997117
3,"DUPLEX U/S ART/BYPS GRAFTS UPP LMB, UNI",DUPLEX U/S ART/BYPS GRAFTS UPP LMB UNI,0.996361,GLUCOSIDASE BETA ACID (GBA) GENE VARIANT ANALY...,0.900581
4,"CONDUCTN STUD, EMG SGL FIBRES NRV & MUSC",CONDUCTN STUD EMG SGL FIBRES NRV & MUSC,0.996189,DILUTE RUSSELL VIPER VENOM TIME (DRVVT),0.916365


In [122]:
sbs_dict = dict(zip(sbs['Short Description'], sbs['Long Description']))
len(sbs)

10081

In [126]:
# Create a boolean mask: True where pair matches the dictionary
mask = common_similarity.apply(
    lambda row: sbs_dict.get(row['SBS_SHORT_DESCRIPTION'], None) == row['SBS_LONG_DESCRIPTION'],
    axis=1
)

matched_common_similarity = common_similarity[mask].copy()

In [127]:
matched_common_similarity.shape

(602, 5)

In [131]:
matched_common_similarity.to_excel('bupa_mapped_same_short_and_long_services_bert.xlsx')

In [136]:
unmatched_common_similarity = common_similarity[~common_similarity['AHJ_DESCRIPTION'].isin(list(matched_common_similarity['AHJ_DESCRIPTION'].unique()))]
unmatched_common_similarity.shape

(1882, 5)

In [137]:
unmatched_common_similarity.to_excel('bupa_mapped_different_short_and_long_services_bert.xlsx')